In [ ]:
from ipyleaflet import Map, Marker, basemaps
from ipyleaflet.velocity import Velocity

import os
import xarray as xr

In [ ]:
ds = xr.open_dataset('data/nc_files/wind_2020_08_27.nc')
print(ds)

In [ ]:
# reshape to 2D
ds = ds.isel(time=0)
ds = ds.rename({'longitude': 'lon', 'latitude': 'lat'})
print(ds)

In [ ]:
center = (44.304655, 5.506011)
zoom = 5

my_map = Map(center=center,
    zoom=zoom,
    interpolation='nearest',
    basemap=basemaps.CartoDB.DarkMatter)

# Add winds to the map

wind = Velocity(data=ds,
                zonal_speed='u10',
                meridional_speed='v10',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=20)
my_map.add_layer(wind)

marker = Marker(location=center, draggable=False)
my_map.add_layer(marker)

my_map

In [ ]:
# save map to html
my_map.save('output/maps/wind_2020_08_27.html')

In [ ]:
if not os.path.exists('wind-global.nc'):
  url = 'https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc'
  import requests
  r = requests.get(url)
  wind_data = r.content
  with open('wind-global.nc', 'wb') as f:
      f.write(wind_data)

center = [0, 0]
zoom = 1
m = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.CartoDB.DarkMatter)

ds = xr.open_dataset('wind-global.nc')
display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
wind = Velocity(data=ds,
                zonal_speed='u_wind',
                meridional_speed='v_wind',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=20,
                display_options=display_options)
m.add_layer(wind)

m

In [ ]:
print(ds)

In [ ]:
ds.attrs